# Лабораторная работа №4 по курсу<br>"Искусственный Интеллект и Машинное Обучение"

# Задание

Найти наиболее информативные признаки для данных вашего варианта.<br>
Произвести оценки зависимости точности модели и числа выбранных признаков.<br>
Используйте полученные ранее знания, такие как разбиение наборов данных и кросс-валидация.

## Результаты, которые необходимо получить в итоге:

1. DataFrame с признаками и их информативностью, отсортированный по убыванию.
2. Графики, показывающие результат оценки зависимости точности модели и числа выбранных признаков.
3. Таблица содержащая f1-score, precision, и recall полученные для лучшей модели, которую вы смогли обучить.
4. Визуализируйте предсказания вашей лучшей модели.

Данные доступны по следующей ссылке:
https://beryl.ssau.ru/nextcloud/index.php/s/62ss9Pa9FdpECrt

# Подготовка данных

Подключим необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
from numpy import load
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

Загрузим содержимое файлов, согласно варианта заданий.<br>
Для упрощени и ускорения загрузки воспользуемся гугл диском.<br>
Для для работы "ноутбука", необходимо чтобы файлы лежали в домашней директории гугл-диска.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
specter = load('/content/drive/MyDrive/14.npy')
mask = load('/content/drive/MyDrive/14_mask.npy')

Проверим размерность данных

In [ ]:
specter.shape, mask.shape

Создадим список всех возможных позиций (y, x) для гиперспектра

In [ ]:
height = specter.shape[0]
width = specter.shape[1]
ordered_positions = np.array(np.meshgrid(np.arange(height), np.arange(width))).T.reshape(-1, 2)

Перемешаем и разделим эти позиции

In [ ]:
train_indices, test_indices = train_test_split(ordered_positions, train_size=0.05, shuffle=True)

Произведем разделение на выборки: тренировочные и тестовые

In [ ]:
x_train = []
y_train = []
x_test = []
y_test = []

for idx in train_indices:
    x_train.append(specter[idx[0], idx[1], ::])
    y_train.append(mask[idx[0], idx[1]])    
    
for idx in test_indices:
    x_test.append(specter[idx[0], idx[1], ::])
    y_test.append(mask[idx[0], idx[1]])    
    
x_train = np.array(x_train, copy=False)
y_train = np.array(y_train, copy=False)
x_test = np.array(x_test, copy=False)
y_test = np.array(y_test, copy=False)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

Произведем масштабирование данных

In [ ]:
mms = MinMaxScaler(feature_range=(1, 10))
x_test = mms.fit_transform(x_test,  y_test)
x_train = mms.transform(x_train)

# <center> Результат 1. DataFrame с признаками и их информативностью, отсортированный по убыванию. </center>

## kBest

Инициализируем и обучим модель kBest

In [ ]:
modelKBest = SelectKBest(score_func=chi2, k=20)
modelKBest = modelKBest.fit(x_train, y_train)

In [ ]:
best_featuresKBest = pd.DataFrame(data=modelKBest.scores_, columns=['chi2_score'])
best_featuresKBest.nlargest(20, 'chi2_score')

## ExtraTreesClassifier

Инициализируем и обучим модель ExtraTreesClassifier

In [ ]:
modelExtraTrees = ExtraTreesClassifier(random_state=1)
modelExtraTrees = modelExtraTrees.fit(x_train, y_train)

In [ ]:
best_featuresExtraTrees = pd.DataFrame(data=modelExtraTrees.feature_importances_, columns=['feature_importance'])
best_featuresExtraTrees.nlargest(20, 'feature_importance')

# <center> Результат 2. Графики, показывающие результат оценки зависимости точности модели и числа выбранных признаков. </center>

## kBest


Произведем построение модели, на основе которой будем строить график

In [ ]:
list_Number_features_kBest = list(range(1, 21))
list_accuracy_kBest = []

# произведем оценку точности для каждого признака
for number in list_Number_features_kBest:
  # произведем отбор определенного количества признаков для дальнейшей тренировки модели
  features_kBest = list(best_featuresKBest.nlargest(number, 'chi2_score').index)
  accuracy_kBest = 0
  iter_count_kBest = 3

  # произведем расчет средней точности по экспериментам на каждой итерации 
  for i in range(iter_count_kBest):
    model = MLPClassifier(max_iter=400, random_state=(2*i+1)*10)
    # обучим модель на только что выбранных объектах
    model.fit(x_train[:,features_kBest], y_train)
    # произведем предсказание на основе тестового набора данных для только что выбранных признаков
    y_pred = model.predict(x_test[:,features_kBest])
    # произведем оценку точности на текущей итерации
    accuracy_kBest += accuracy_score(y_test, y_pred)

  accuracy_kBest /= iter_count_kBest
  list_accuracy_kBest.append(accuracy_kBest)

Построим график

In [ ]:
plt.plot(list_Number_features_kBest, list_accuracy_kBest, 'o-')
plt.grid()
plt.xlabel('number of features k Best')
plt.ylabel('accuracy k Best')
plt.title('model k Best')
plt.show()

## ExtraTreesClassifier

Произведем построение модели, на основе которой будем строить график

In [ ]:
list_Number_features_ExtraTrees = list(range(1, 21))
list_accuracy_ExtraTrees = []

# произведем оценку точности для каждого признака
for number in list_Number_features_ExtraTrees:
  # произведем отбор определенного количества признаков для дальнейшей тренировки модели
  features_ExtraTrees = list(best_featuresExtraTrees.nlargest(number, 'feature_importance').index)
  accuracy_ExtraTrees = 0
  iter_count_ExtraTrees = 3

  # произведем расчет средней точности по экспериментам на каждой итерации 
  for i in range(iter_count_ExtraTrees):
    model = MLPClassifier(max_iter=400, random_state=(2*i+1)*10)
    # обучим модель на только что выбранных объектах
    model.fit(x_train[:,features_ExtraTrees], y_train)
    # произведем предсказание на основе тестового набора данных для только что выбранных признаков
    y_pred = model.predict(x_test[:,features_ExtraTrees])
    # произведем оценку точности на текущей итерации
    accuracy_ExtraTrees += accuracy_score(y_test, y_pred)

  accuracy_ExtraTrees /= iter_count_ExtraTrees
  list_accuracy_ExtraTrees.append(accuracy_ExtraTrees)

Построим график

In [ ]:
plt.plot(list_Number_features_ExtraTrees, list_accuracy_ExtraTrees, 'o-')
plt.grid()
plt.xlabel('number of features Extra Trees')
plt.ylabel('accuracy Extra Trees')
plt.title('model Extra Trees')
plt.show()

## Выбор лучшей модели

Рассматривая графики, соершенно точно можно сказать, что лучшей моделью будет являться<br>***ExtraTreesClassifier***<br>
собственно с ней мы и продолжим работу.

# <center> Результат 3. Таблица содержащая f1-score, precision, и recall полученные для лучшей модели, которую вы смогли обучить. </center>

In [ ]:
best_model = MLPClassifier(max_iter=400, random_state=0)

parameters = {
    'hidden_layer_sizes': [(50,), (100,), (150,)]
}

metrics = ['precision_macro', 'recall_macro', 'f1_macro']

search = GridSearchCV(
    estimator=best_model,
    param_grid=parameters,
    scoring=metrics,
    refit='precision_macro'
)

In [ ]:
tventy_best_featuresExtraTrees = list(best_featuresExtraTrees.nlargest(20, 'feature_importance').index)
search.fit(x_train[:,tventy_best_featuresExtraTrees], y_train)

In [ ]:
results = search.cv_results_
results_df = pd.DataFrame(results)
results_df[['mean_test_precision_macro', 'mean_test_recall_macro', 'mean_test_f1_macro']]

# <center> Результат 4. Визуализируйте предсказания вашей лучшей модели. </center>

In [ ]:
best_model = search.best_estimator_

In [ ]:
train_preds = best_model.predict(x_train[:,tventy_best_featuresExtraTrees])
test_preds = best_model.predict(x_test[:,tventy_best_featuresExtraTrees])

In [ ]:
predictions_map = np.zeros_like(mask)

In [ ]:
for (y, x), pred in zip(train_indices, train_preds):
    predictions_map[y, x] = pred
    
for (y, x), pred in zip(test_indices, test_preds):
    predictions_map[y, x] = pred

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 10))
sns.heatmap(mask, ax=ax1)
sns.heatmap(predictions_map, ax=ax2)
ax1.set_title('Mask')
ax2.set_title('Predictions')